In [ ]:
# Importing libraries
import os
import cv2
from PIL import Image

# Set path to the Google Drive folder with images
path = "/content/sample_data/Weeds2"
os.chdir(path)

mean_height = 0
mean_width = 0

# Collect images with proper extensions
valid_exts = (".jpg", ".jpeg", ".png", ".JPG", ".JPEG", ".PNG")
images = [file for file in os.listdir(path) if file.endswith(valid_exts)]

num_of_images = len(images)
print("Number of Images:", num_of_images)

if num_of_images == 0:
    raise ValueError(f"⚠️ No images found in folder: {path}")

# Calculating mean width and height
for file in images:
    im = Image.open(os.path.join(path, file))
    width, height = im.size
    mean_width += width
    mean_height += height

mean_width = int(mean_width / num_of_images)
mean_height = int(mean_height / num_of_images)

# Resizing all images
for file in images:
    im = Image.open(os.path.join(path, file))
    im_resized = im.resize((mean_width, mean_height), Image.LANCZOS)
    im_resized.save(file, "JPEG", quality=95)
    print(f"{file} resized")

# Function to generate video
def generate_video():
    video_name = "mygeneratedvideo.avi"

    # Sort images for proper sequence
    images_sorted = sorted(
        [img for img in os.listdir(path) if img.endswith(valid_exts)]
    )

    print("Images:", images_sorted)

    if len(images_sorted) == 0:
        print("⚠️ No images found, cannot generate video.")
        return

    # Get frame size from first image
    frame = cv2.imread(os.path.join(path, images_sorted[0]))
    height, width, layers = frame.shape

    # Video writer.
    video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*"DIVX"), 10, (width, height))

    # Append frames
    for image in images_sorted:
        img = cv2.imread(os.path.join(path, image))
        if img is not None:
            video.write(img)

    video.release()
    cv2.destroyAllWindows()
    print("✅ Video generated successfully:", video_name)

# Run function
generate_video()


In [ ]:
from google.colab import files

# Download the video
files.download("mygeneratedvideo.avi")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#Classification of an image testimony code

import cv2
import os
from skimage.metrics import structural_similarity as ssim

def classify_image(image_path, video_path, threshold=0.7):
    # Load input image
    input_img = cv2.imread(image_path)
    input_img = cv2.cvtColor(input_img, cv2.COLOR_BGR2RGB)

    # Open video
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print("Total frames in video:", frame_count)

    best_score = -1  # highest SSIM
    frame_index = -1

    for i in range(frame_count):
        ret, frame = cap.read()
        if not ret:
            break
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Resize input to match frame
        resized_input = cv2.resize(input_img, (frame_rgb.shape[1], frame_rgb.shape[0]))

        # Compute SSIM similarity
        score, _ = ssim(resized_input, frame_rgb, channel_axis=2, full=True)

        if score > best_score:
            best_score = score
            frame_index = i

    cap.release()

    print(f"Best SSIM Score: {best_score:.3f} at frame {frame_index}")

    # Classification decision
    if best_score >= threshold:
        return "✅ The image belongs to the video"
    else:
        return "❌ The image does NOT belong to the video"

# Example usage
image_path = "/content/sample_data/image1.jpg"   # your test image
video_path = "mygeneratedvideo.avi"

result = classify_image(image_path,video_path,threshold=0.7)
result

error: OpenCV(4.12.0) /io/opencv/modules/imgproc/src/color.cpp:199: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
